#### This first cell is entirely creating jupyter widgets/reading ABoVE vector data. We can hide it from the users like Yaxing did with his workshop. Then show a walkthrough of the important pieces at the bottom of the notebook:

In [31]:
from CMR import *
from QueryCMR import *


above_search = collections.keyword("*Boreal Vulnerability Experiment*").get_all()
above_results = [r for r in above_results if any([
    "ABoVE" in r["dataset_id"],
    "ABoVE" in r["summary"]])
]

In [32]:
len(above_results)

45

In [29]:
above_results2[0]

{'processing_level_id': '4',
 'boxes': ['39.42 -176.12 81.61 -66.92'],
 'time_start': '2014-01-01T00:00:00.000Z',
 'version_id': '1',
 'updated': '2019-04-15T18:55:48.000Z',
 'dataset_id': 'A Concise Experiment Plan for the Arctic-Boreal Vulnerability Experiment',
 'has_spatial_subsetting': False,
 'has_transforms': False,
 'has_variables': False,
 'data_center': 'ORNL_DAAC',
 'short_name': 'ABoVE_Concise_Experiment_Plan_1617',
 'organizations': ['ORNL_DAAC'],
 'title': 'A Concise Experiment Plan for the Arctic-Boreal Vulnerability Experiment',
 'coordinate_system': 'CARTESIAN',
 'summary': "This document presents the Concise Experiment Plan for NASA's Arctic-Boreal Vulnerability Experiment (ABoVE) to serve as a guide to the Program as it identifies the research to be conducted under this study. Research for ABoVE will link field-based, process-level studies with geospatial data products derived from airborne and satellite remote sensing, providing a foundation for improving the analys

In [30]:
for r in above_results2:
    print(r["short_name"])

ABoVE_Concise_Experiment_Plan_1617
ABoVE_reference_grid_v2_1527
ABoVE_Airborne_AVIRIS_NG_1569
Arctic_Winter_Respiration_1620
Erosion_Vegetation_Yukon_1616
ABoVE_Arctic_CAP_1658
ABoVE_PBand_SAR_1657
ABoVE_Thaw_Depth_1579
Alaska_Yukon_NDVI_1614
ABoVE_AirSWOT_Radar_Data_1646
ABoVE_Open_Water_Map_1643
Wildfire_Effects_Spruce_Field_1595
ABLVIS2
ABLVIS1B
ABoVE_Planning_Field_Sites_1582
Snowpack_Dall_Sheep_Track_1583
AK_North_Slope_NEE_CH4_Flux_1562
Rain-on-Snow_Data_1611
Maps_AGB_North_Slope_AK_1565
Wildfires_NWT_Canada_1548
Fire_Emissions_NWT_1561
Permafrost_Thaw_Depth_YK_1598
Last_Day_Spring_Snow_1528
ABoVE_Fire_Severity_dNBR_1564
Seasonality_Tundra_Vegetation_1606
Wildfires_Date_of_Burning_1559
ABoVE_Frac_Open_Water_1362
Alaskan_Wildfire_C_Emissions_1341
Arctic_Boreal_Burned_Area_1526
Dall_Sheep_Population_Dynamics_1640
MODIS_CCaN_NDVI_Trends_Alaska_1666
Wildfires_2014_NWT_Canada_1307
ABoVE_Soil_Radiocarbon_NWT_1664
Canada_Boreal_Forest_Greenness_1587
ReSALT_ALT_GPR_1265
ALT_GPR_Barrow_13

In [2]:
import json
import ipyleaflet as pl
import ipywidgets as pw

# ------------------------------------------------------------------------------- 
# the above grid (not using C because too fine)

grid_json = 'data/ABoVE_240m_30m_5m_grid_tiles/ABoVE_240m_30m_5m_grid_tiles.json'
grid_layers = {"A": pl.LayerGroup(), "B": pl.LayerGroup()}
grid_table = {"A": {}, "B": {}, "C": {}}

with open(grid_json, 'r') as file:
    above_grid = json.load(file)
    
    for feat in above_grid["features"]:
        prop = feat["properties"]          # collect grid cell properties
        level = prop["grid_level"]         # get level A, B, or C
        feat["properties"]["style"] = {
            "weight": 0.75,
            "color": "aliceblue",
            "fillColor": "#FFFFFF",
            "fillOpacity": 0.4}
        
        if level!="C": 
        
            id = prop["grid_id"]               # get grid cell id
            cell = pl.GeoJSON(data=feat)       # make geojson map layer

            # add object to map group and reference dictionary
            grid_table[level][id] = cell
            grid_layers[level].add_layer(cell)

    
# ------------------------------------------------------------------------------- 
# the above domain

domain_json = 'data/ABoVE_Study_Domain/ABoVE_Study_Domain.json'  
with open(domain_json, 'r') as file:
    above_domain = json.load(file)
    above_domain["features"][0]["properties"]["style"] = {
        "weight": 0.75,
        "color": "#FFFFFF",
        "fillColor": "#FFFFFF",
        "fillOpacity": 0}
    domain_layer = pl.GeoJSON(data=above_domain)

    
# -------------------------------------------------------------------------------
# JSON input interface

# the default json string for selecting collections/granules 
with open('data/geom.json', 'r') as file:
    default_json = file.read().replace('\n', '')

geojson_label = pw.HTML("<h4><b>Paste your GeoJSON: </b></h4>")
geojson_text = pw.Textarea(
    value=default_json,
    placeholder='Paste GeoJSON string here.',
    disabled=False,
    layout=pw.Layout(width="auto", height="200px"))


# -------------------------------------------------------------------------------
# map widget interface

# load a basemap layer from ESRI service
esri = pl.basemap_to_tiles(pl.basemaps.Esri.WorldImagery)
gibs = pl.basemap_to_tiles(pl.basemaps.NASAGIBS.ModisTerraTrueColorCR, "2019-01-01")

# initialize map widget
m = pl.Map(layers=(esri, domain_layer,), center=(65, -150), zoom=5, scroll_wheel_zoom=True)
control = pl.SplitMapControl(left_layer=grid_layers["A"], right_layer=grid_layers["B"])
m.add_control(control)

# add an options menu
#m.add_control(pl.LayersControl())

# -------------------------------------------------------------------------------
# display!
pw.VBox([geojson_label, geojson_text, m])

In [ ]:
above_grid["features"][0]

# TOC:

[CMR: find ABoVE datasets](above-ornl-daac.ipynb#Searching-CMR-with-Python)

# Subset, download ABoVE data from THREDDS: 

In [ ]:
geojson_text.value

In [ ]:
import json
import xarray as xr
from osgeo import gdal, ogr, osr

#jsonstr = geojson_text.value
shp = ogr.Open("geo.shp")    # open shapefile
lyr = shp.GetLayer()             # get first (and only) layer
ext = lyr.GetExtent()            # get extent

ext

In [ ]:
source_srs = lyr.GetSpatialRef()                                 # get srs
target_srs = osr.SpatialReference()                              # output srs
target_srs.ImportFromEPSG(4326)                            # import daymet proj

transform = osr.CoordinateTransformation(source_srs, target_srs) # the transformation

In [ ]:
feat = lyr.GetFeature(0)         # select the input layer's only feature
geom = feat.GetGeometryRef()     # get the feature's geometry
geom.Transform(transform)          # transform to daymet lcc

geom

In [ ]:
memvector = ogr.GetDriverByName('Memory').CreateDataSource('mem')  # open OGR vector memory dataset
memlyr = memvector.CreateLayer('poly', srs=target_srs)             # make a dummy layer in daymet lcc
memfeat = ogr.Feature(memlyr.GetLayerDefn())                       # init a dummy feature
memfeat.SetGeometryDirectly(geom)                                  # set transformed geom to feature
memlyr.CreateFeature(memfeat)                                      # add dummy feature to layer

lccext = memlyr.GetExtent()                                        # get the lcc extent for the next step
lccext

In [ ]:

above_thredds = ("https://thredds.daac.ornl.gov/thredds/catalogs/ornldaac/"
                 "Field_Campaign_Data/Arctic-Boreal_Vulnerability_Experiment/"
                 "Arctic-Boreal_Vulnerability_Experiment.html")

response = requests.get(above_thredds)

from lxml import html

page = html.fromstring(response.content)  # parse html to list
href = page.xpath('//a/@href')                              # get links
links = [link for link in href] # select links 

links

# Searching CMR with Python:

A Python wrapper to NASA's Common Metadata Repository (CMR) API is available through pip:
```{shell}
pip install python-cmr
```

See what's available in `cmr` package:

In [ ]:
import cmr
import json

printd = lambda o: print([a for a in dir(o) if "__" not in a])
#help(cmr)

Import the public classes; check out collections:

In [ ]:
from cmr import CollectionQuery, GranuleQuery

collections = CollectionQuery()
granules = GranuleQuery()

printd(collections)

Get ORNL DAAC collections:

In [ ]:
ornldaac = collections.archive_center("ORNL_DAAC")

printd(ornldaac)

### Get all ORNL DAAC's ABoVE datasets in CMR. Print the number of datasets and the first dataset's record:

In [ ]:
above = [d for d in ornldaac.keyword("ABoVE").get_all() if "ABoVE" in d["title"]]
print("Number of ABoVE datasets at ORNL DAAC:"+str(len(above)))
print(print(json.dumps(above[0], indent=4)))

### Datasets to pandas data frame:

In [ ]:
import pandas as pd

datasets = pd.DataFrame(above)

datasets.head(5)

### List of dataset titles:

In [ ]:
dsnames = datasets.dataset_id.tolist()
dsnames

### Get last 10 published ABoVE datasets:

In [ ]:
datasets.sort_values("updated")

In [ ]:
cmr = "https://cmr.earthdata.nasa.gov/search/"

project = "ABoVE"



collections = "collections?" #  options:
projectstr = "project=%s"




project % "ABoVE"